In [27]:
import pandas as pd
import numpy as np 
import pickle
import pickle5
import matplotlib.pyplot as plt
import datetime
from pysurvival.models.semi_parametric import CoxPHModel
from pysurvival.utils.display import display_loss_values
from sklearn.preprocessing import StandardScaler
from pysurvival.utils.display import compare_to_actual
from pysurvival.utils import save_model, load_model
from pysurvival.utils.sklearn_adapter import sklearn_adapter
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import mean_squared_error
from sklearn.metrics import auc, r2_score
from sklearn.utils import resample
from lifelines.utils import concordance_index
from sklearn import metrics
## block warning
import warnings
warnings.filterwarnings('ignore')

In [30]:
with open("../data/ut_V6_classification-4H.pkl", "rb") as fh:
    df = pickle5.load(fh)
    
df.head(3)

,life_time,datetime_start,datetime_end,latitude,longitude,distance_center_km,weekday_b,charging_ports,lag1,lag2,...,hour_b_22,hour_b_23,Station_Name_BOULDER / N BOULDER REC 1,Station_Name_COMM VITALITY / 1000WALNUT,Station_Name_COMM VITALITY / 1104 SPRUCE1,4Hsplit,y_pred,Station_Name,weekday_b_name,tod
3734,118.5,2018-01-05 14:25:00,2018-01-05 16:23:30,40.000148,-105.282437,2.096847,4,2,1271.5,135.5,...,0,0,0,0,0,0,1,BOULDER / BASELINE ST1,Friday,Midday
3735,886.5,2018-01-05 17:02:00,2018-01-06 07:48:30,40.000148,-105.282437,2.096847,4,2,118.5,1271.5,...,0,0,0,0,0,1,1,BOULDER / BASELINE ST1,Friday,Afternoon
3736,326.5,2018-01-06 09:28:00,2018-01-06 14:54:30,40.000148,-105.282437,2.096847,5,2,886.5,118.5,...,0,0,0,0,0,1,0,BOULDER / BASELINE ST1,Saturday,Morning


### Usefull functions

In [31]:
## Get point predictions
def point_pred_single(model, X_test):
    T_pred = []
    # Get survival curves
    cph_pred = model.predict_survival(X_test)
    # get times of survival prediction
    time = model.times
    # test
    for i in range(0,len(cph_pred)):
        T_pred.append(auc(time,cph_pred[i]))
    
    return T_pred

In [32]:
## Get point predictions
def point_pred(model, X_test, X_train):
    T_pred = []
    T_pred_train = []
    # Get survival curves
    cph_pred = model.predict_survival(X_test)
    cph_pred_train = model.predict_survival(X_train)
    # get times of survival prediction
    time = model.times
    # test
    for i in range(0,len(cph_pred)):
        T_pred.append(auc(time,cph_pred[i]))
    # train
    for i in range(0,len(cph_pred_train)):
        T_pred_train.append(auc(time,cph_pred_train[i]))
    
    return T_pred, T_pred_train

In [33]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true))*100

In [34]:
## To test model
def test_model(y_test, y_pred):
    MAE = metrics.mean_absolute_error(y_test, y_pred)
    #print('MAE (Mean Absolute Error):', MAE)
    MSE = metrics.mean_squared_error(y_test, y_pred)
    #print('MSE (Mean Squared Error):', MSE)
    RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    #print('RMSE (Root Mean Squared Error):', RMSE)
    MAPE = mean_absolute_percentage_error(y_test, y_pred)
    
    NRMSE = RMSE/np.mean(y_test)
    return MAE, RMSE, MSE, MAPE, NRMSE

## Modeling features

To make coding easier the names of the models are changed:
- M1: Baseline
- M5: Full

In [35]:
time_column = 'life_time'
event_column = 'event'

## M1
features1 = ['weekday_b_name_Monday', 'weekday_b_name_Saturday',
             'weekday_b_name_Sunday', 'weekday_b_name_Thursday',
             'weekday_b_name_Tuesday', 'weekday_b_name_Wednesday', 'tod_Evening', 'tod_Midday', 'tod_Morning',
       'Station_Name_BOULDER / N BOULDER REC 1',
       'Station_Name_COMM VITALITY / 1000WALNUT',
       'Station_Name_COMM VITALITY / 1104 SPRUCE1'] # dow + tod

## M5
features5_cat = ['weekday_b_name_Monday', 'weekday_b_name_Saturday',
             'weekday_b_name_Sunday', 'weekday_b_name_Thursday',
             'weekday_b_name_Tuesday', 'weekday_b_name_Wednesday','tod_Evening', 'tod_Midday', 'tod_Morning',
       'Station_Name_BOULDER / N BOULDER REC 1',
       'Station_Name_COMM VITALITY / 1000WALNUT',
       'Station_Name_COMM VITALITY / 1104 SPRUCE1']
features5_con = ['lag1', 'lag2', 'lag3','near_charge_time_4H', 'near_charge_energy_4H',
                 'charge_time_4H', 'charge_energy_4H'] # dow + agg. tod + lag + activ.

In [36]:
#data split
split = 0.8

In [37]:
df['event'] = np.ones(len(df))

In [38]:
train, test = np.split(df, [int(split * len(df))])
print("Training shape:",train.shape)
print("Testing shape:", test.shape)

Training shape: (4088, 103)
Testing shape: (1022, 103)


In [39]:
trainS = train[train['y_pred'] == 0]
trainL = train[train['y_pred'] == 1]

testS = test[test['y_pred'] == 0]
testL = test[test['y_pred'] == 1]

In [40]:
print("SHORT: Train: {} Test: {}".format(trainS.shape[0], testS.shape[0]))
print("LONG: Train: {} Test: {}".format(trainL.shape[0], testL.shape[0]))

SHORT: Train: 2433 Test: 629
LONG: Train: 1655 Test: 393


In [42]:
## Get X 
# Baseline
X1_S = trainS[features1]
X1_L = trainL[features1]

## Full
# Short
X5_S = trainS[features5_cat+features5_con]
scaler = StandardScaler()
X5_S[features5_con] = scaler.fit_transform(X5_S[features5_con])
X5_S_test = testS[features5_cat+features5_con]
X5_S_test[features5_con] = scaler.transform(X5_S_test[features5_con])
# Full
X5_L = trainL[features5_cat+features5_con]
scaler = StandardScaler()
X5_L[features5_con] = scaler.fit_transform(X5_L[features5_con])
X5_L_test = testL[features5_cat+features5_con]
X5_L_test[features5_con] = scaler.transform(X5_L_test[features5_con])

## Get E -train
E_S = trainS[event_column]
E_L = trainL[event_column]

## Get T - train
T_S = trainS['life_time']
T_L = trainL['life_time']

## Get E -Test
E_S_test = testS[event_column]
E_L_test = testL[event_column]

## Get T - Test
T_S_test = testS['life_time']
T_L_test = testL['life_time']

### Find hyperparameters

In [16]:
learning_rates = [0.001, 0.01, 0.1]
regularization = [0.01, 0.1, 1.0]

Short:

In [19]:
##############################################################################
### Find most optimal parameters for every station
best_c = -1
best_lear = None
best_regu = None
best_model = None
fails = {}

total_it = len(learning_rates)*len(regularization)
i=0

# loop over parameters
for lear in learning_rates:
    for regu in regularization:
        try: 
            cph = CoxPHModel()
            cph.fit(X1_S, T_S, E_S,
                    lr=lear, l2_reg = regu, init_method='zeros', max_iter=800, verbose=False)

            #point pred
            point_predictions = point_pred_single(cph,X1_S)

            # c-index
            c_temp = concordance_index(T_S, point_predictions)

            if c_temp >= best_c:
                print("NEW BEST!!", i)
                best_lear = lear
                best_regu = regu
                best_c = c_temp  
        except:
            print("FAIL, lr:", lear, " l2:", regu)   
        i+=1
        print(round(i/total_it,2), "  ", end='')
print("Parameter tuning done.")

NEW BEST!! 0
0.11   0.22   0.33   NEW BEST!! 3
0.44   NEW BEST!! 4
0.56   0.67   The gradient exploded.You should reduce the learning rate (lr) or try a different initialization.
FAIL, lr: 0.1  l2: 0.01
0.78   The gradient exploded.You should reduce the learning rate (lr) or try a different initialization.
FAIL, lr: 0.1  l2: 0.1
0.89   1.0   Parameter tuning done.


In [20]:
lr_s1 = best_lear
l2_reg_s1 = best_regu

print("SHORT, M1: lr: {} l2_reg: {}".format(best_lear,best_regu))

LONG, M1: lr: 0.01 l2_reg: 0.1


In [21]:
##############################################################################
### Find most optimal parameters for every station
best_c = -1
best_lear = None
best_regu = None
best_model = None
fails = {}

total_it = len(learning_rates)*len(regularization)
i=0

# loop over parameters
for lear in learning_rates:
    for regu in regularization:
        try: 
            cph = CoxPHModel()
            cph.fit(X5_S, T_S, E_S,
                    lr=lear, l2_reg = regu, init_method='zeros', max_iter=800, verbose=False)

            #point pred
            point_predictions = point_pred_single(cph,X5_S)

            # c-index
            c_temp = concordance_index(T_S, point_predictions)

            if c_temp >= best_c:
                print("NEW BEST!!", i)
                best_lear = lear
                best_regu = regu
                best_c = c_temp  
        except:
            print("FAIL, lr:", lear, " l2:", regu)   
        i+=1
        print(round(i/total_it,2), "  ", end='')
print("Parameter tuning done.")

NEW BEST!! 0
0.11   NEW BEST!! 1
0.22   NEW BEST!! 2
0.33   NEW BEST!! 3
0.44   0.56   0.67   The gradient exploded.You should reduce the learning rate (lr) or try a different initialization.
FAIL, lr: 0.1  l2: 0.01
0.78   The gradient exploded.You should reduce the learning rate (lr) or try a different initialization.
FAIL, lr: 0.1  l2: 0.1
0.89   1.0   Parameter tuning done.


In [22]:
lr_s5 = best_lear
l2_reg_s5 = best_regu

print("SHORT, M5: lr: {} l2_reg: {}".format(lr_s5,l2_reg_s5))

LONG, M1: lr: 0.01 l2_reg: 0.01


Long:

In [23]:
##############################################################################
### Find most optimal parameters for every station
best_c = -1
best_lear = None
best_regu = None
best_model = None
fails = {}

total_it = len(learning_rates)*len(regularization)
i=0

# loop over parameters
for lear in learning_rates:
    for regu in regularization:
        try: 
            cph = CoxPHModel()
            cph.fit(X1_L, T_L, E_L,
                    lr=lear, l2_reg = regu, init_method='zeros', max_iter=800, verbose=False)

            #point pred
            point_predictions = point_pred_single(cph,X1_L)

            # c-index
            c_temp = concordance_index(T_L, point_predictions)

            if c_temp >= best_c:
                print("NEW BEST!!", i)
                best_lear = lear
                best_regu = regu
                best_c = c_temp  
        except:
            print("FAIL, lr:", lear, " l2:", regu)   
        i+=1
        print(round(i/total_it,2), "  ", end='')
print("Parameter tuning done.")

NEW BEST!! 0
0.11   NEW BEST!! 1
0.22   NEW BEST!! 2
0.33   0.44   0.56   0.67   0.78   0.89   1.0   Parameter tuning done.


In [24]:
lr_l1 = best_lear
l2_reg_l1 = best_regu

print("LONG, M1: lr: {} l2_reg: {}".format(best_lear,best_regu))

LONG, M1: lr: 0.001 l2_reg: 1.0


In [25]:
##############################################################################
### Find most optimal parameters for every station
best_c = -1
best_lear = None
best_regu = None
best_model = None
fails = {}

total_it = len(learning_rates)*len(regularization)
i=0

# loop over parameters
for lear in learning_rates:
    for regu in regularization:
        try: 
            cph = CoxPHModel()
            cph.fit(X5_L, T_L, E_L,
                    lr=lear, l2_reg = regu, init_method='zeros', max_iter=800, verbose=False)

            #point pred
            point_predictions = point_pred_single(cph,X5_L)

            # c-index
            c_temp = concordance_index(T_L, point_predictions)

            if c_temp >= best_c:
                print("NEW BEST!!", i)
                best_lear = lear
                best_regu = regu
                best_c = c_temp  
        except:
            print("FAIL, lr:", lear, " l2:", regu)   
        i+=1
        print(round(i/total_it,2), "  ", end='')
print("Parameter tuning done.")

NEW BEST!! 0
0.11   0.22   NEW BEST!! 2
0.33   NEW BEST!! 3
0.44   NEW BEST!! 4
0.56   0.67   0.78   NEW BEST!! 7
0.89   1.0   Parameter tuning done.


In [26]:
lr_l5 = best_lear
l2_reg_l5 = best_regu

print("LONG, M5: lr: {} l2_reg: {}".format(best_lear,best_regu))

LONG, M1: lr: 0.1 l2_reg: 0.1


Specify the hyperparameters.

In [43]:
# dictionary to save results in 
results = {}

# M1
lr_s1 = 0.01
l2_reg_s1 = 0.1
lr_l1 = 0.001
l2_reg_l1 = 1.0
# M5
lr_s5 = 0.01
l2_reg_s5 = 0.01
lr_l5 = 0.1
l2_reg_l5 = 0.1


#### CPH - Model 1

In [44]:
#########################################################
#################### SEPERATE RESULTS ###################

results['M1'] = {}
results['M1']['S'] = {}
results['M1']['L'] = {}
results['M1']['Com'] = {}

################ Fit models ################
#
####################### short #######################
regs = CoxPHModel()
regs.fit(X1_S, T_S, E_S, lr=lr_s1, l2_reg = l2_reg_s1, 
        init_method='zeros', max_iter=800, verbose=False)

## Point predictions
T_test_predS, T_train_predS = point_pred(regs, X1_S_test, X1_S)

## Test model
c_test = concordance_index(T_S_test, T_test_predS)
c_train = concordance_index(T_S, T_train_predS)
print(c_test)

## Get metrics
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(T_S_test,T_test_predS)
MAE_train, RMSE_train, MSE_train, MAPE_train, NRMSE_train = test_model(T_S,T_train_predS) 

## Save results
results['M1']['S'] = {'RMSE_train':RMSE_train,'RMSE_test':RMSE_test,
                      'MAE_train': MAE_train,'MAE_test':MAE_test,
                      'R2_test':r2_score(T_S_test,T_test_predS), 
                      'R2_train':r2_score(T_S,T_train_predS),
                      'MAPE_test':MAPE_test,'MAPE_train':MAPE_train,
                      'NRMSE_test':NRMSE_test,'NRMSE_train':NRMSE_train,
                      'c-train':c_train,'c-test':c_test}


####################### long #######################
regl = CoxPHModel()
regl.fit(X1_L, T_L, E_L, lr=lr_l1, l2_reg = l2_reg_l1, 
        init_method='zeros', max_iter=800, verbose=False)

## Point predictions
T_test_predL, T_train_predL = point_pred(regl, X1_L_test, X1_L)

## Test model
c_test = concordance_index(T_L_test, T_test_predL)
c_test = concordance_index(T_L, T_train_predL)
print(c_test)

## Get metrics
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(T_L_test,T_test_predL)
MAE_train, RMSE_train, MSE_train, MAPE_train, NRMSE_train = test_model(T_L,T_train_predL) 

## Save results
results['M1']['L'] = {'RMSE_train':RMSE_train,'RMSE_test':RMSE_test,
                      'MAE_train': MAE_train,'MAE_test':MAE_test,
                      'R2_test':r2_score(T_L_test,T_test_predL), 
                      'R2_train':r2_score(T_L,T_train_predL),
                      'MAPE_test':MAPE_test,'MAPE_train':MAPE_train,
                      'NRMSE_test':NRMSE_test,'NRMSE_train':NRMSE_train,
                      'c-train':c_train,'c-test':c_test}

################## Combined ##################
T_com_real_test1 = pd.concat([T_S_test, T_L_test])
T_test_predS.extend(T_test_predL)
T_com_pred_test1 = T_test_predS

T_com_real_train = pd.concat([T_S, T_L])
T_train_predS.extend(T_train_predL)
T_com_pred_train = T_train_predS


## Get metrics
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(T_com_real_test1, T_com_pred_test1)
MAE_train, RMSE_train, MSE_train, MAPE_train, NRMSE_train = test_model(T_com_real_train, T_com_pred_train) 

## C-index
c_test = concordance_index(T_com_real_test1, T_com_pred_test1)
c_train = concordance_index(T_com_real_train, T_com_pred_train)

## Save results
results['M1']['Com'] = {'RMSE_train':RMSE_train,'RMSE_test':RMSE_test,
                      'MAE_train': MAE_train,'MAE_test':MAE_test,
                      'R2_test':r2_score(T_com_real_test1,T_com_pred_test1), 
                      'R2_train':r2_score(T_com_real_train,T_com_pred_train),
                      'MAPE_test':MAPE_test,'MAPE_train':MAPE_train,
                      'NRMSE_test':NRMSE_test,'NRMSE_train':NRMSE_train,
                      'c-train':c_train,'c-test':c_test}

0.533285928333799
0.5872982852279234


#### CPH - Model 5

In [45]:
#########################################################
#################### SEPERATE RESULTS ###################

results['M5'] = {}
results['M5']['S'] = {}
results['M5']['L'] = {}
results['M5']['Com'] = {}

################ Fit models ################
#
####################### short #######################
regs = CoxPHModel()
regs.fit(X5_S, T_S, E_S, lr=lr_s5, l2_reg = l2_reg_s5, 
        init_method='zeros', max_iter=800, verbose=False)

## Point predictions
T_test_predS, T_train_predS = point_pred(regs, X5_S_test, X5_S)

## Test model
c_test = concordance_index(T_S_test, T_test_predS)
c_train = concordance_index(T_S, T_train_predS)
print(c_test)

## Get metrics
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(T_S_test,T_test_predS)
MAE_train, RMSE_train, MSE_train, MAPE_train, NRMSE_train = test_model(T_S,T_train_predS) 

## Save results
results['M5']['S'] = {'RMSE_train':RMSE_train,'RMSE_test':RMSE_test,
                      'MAE_train': MAE_train,'MAE_test':MAE_test,
                      'R2_test':r2_score(T_S_test,T_test_predS), 
                      'R2_train':r2_score(T_S,T_train_predS),
                      'MAPE_test':MAPE_test,'MAPE_train':MAPE_train,
                      'NRMSE_test':NRMSE_test,'NRMSE_train':NRMSE_train,
                      'c-train':c_train,'c-test':c_test}

####################### long #######################
regl = CoxPHModel()
regl.fit(X5_L, T_L, E_L, lr=lr_l5, l2_reg = l2_reg_l5, 
        init_method='zeros', max_iter=800, verbose=False)

## Point predictions
T_test_predL, T_train_predL = point_pred(regl, X5_L_test, X5_L)

## Test model
c_test = concordance_index(T_L_test, T_test_predL)
c_train = concordance_index(T_L, T_train_predL)
print(c_test)

## Get metrics
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(T_L_test,T_test_predL)
MAE_train, RMSE_train, MSE_train, MAPE_train, NRMSE_train = test_model(T_L,T_train_predL) 

## Save results
results['M5']['L'] = {'RMSE_train':RMSE_train,'RMSE_test':RMSE_test,
                      'MAE_train': MAE_train,'MAE_test':MAE_test,
                      'R2_test':r2_score(T_L_test,T_test_predL), 
                      'R2_train':r2_score(T_L,T_train_predL),
                      'MAPE_test':MAPE_test,'MAPE_train':MAPE_train,
                      'NRMSE_test':NRMSE_test,'NRMSE_train':NRMSE_train,
                      'c-train':c_train,'c-test':c_test}

## Point predictions
T_test_predL, T_train_predL = point_pred(regl, X5_L_test, X5_L)

################## Combined ##################
T_com_real_test5 = pd.concat([T_S_test, T_L_test])
T_test_predS.extend(T_test_predL)
T_com_pred_test5 = T_test_predS

T_com_real_train = pd.concat([T_S, T_L])
T_train_predS.extend(T_train_predL)
T_com_pred_train = T_train_predS

## Get metrics
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(T_com_real_test5, T_com_pred_test5)
MAE_train, RMSE_train, MSE_train, MAPE_train, NRMSE_train = test_model(T_com_real_train, T_com_pred_train) 

## C-index
c_test = concordance_index(T_com_real_test5, T_com_pred_test5)
c_train = concordance_index(T_com_real_train, T_com_pred_train)

## Save results
results['M5']['Com'] = {'RMSE_train':RMSE_train,'RMSE_test':RMSE_test,
                      'MAE_train': MAE_train,'MAE_test':MAE_test,
                      'R2_test':r2_score(T_com_real_test1,T_com_pred_test1), 
                      'R2_train':r2_score(T_com_real_train,T_com_pred_train),
                      'MAPE_test':MAPE_test,'MAPE_train':MAPE_train,
                      'NRMSE_test':NRMSE_test,'NRMSE_train':NRMSE_train,
                      'c-train':c_train,'c-test':c_test}

0.5341189018970465
0.6001611205675602


### Results

In [50]:
models = ['M1', 'M5']

print("SHORT")
for mo in models:
    r = results[mo]['S']   
    RMSE = r['RMSE_test']
    R2 = r['R2_test']
    MAE = r['MAE_test']
    MAPE = r['MAPE_test']
    NRMSE = r['NRMSE_test']
    C_idx = r['c-test']

    print("DeepSurv {}: R2: {} RMSE: {} C-idx: {} MAE: {} MAPE: {} NRMSE: {}".format(mo,
                                                                          round(R2,2),
                                                                          round(RMSE,0),
                                                                          round(C_idx,2),
                                                                          round(MAE,0),
                                                                          round(MAPE,0),
                                                                          round(NRMSE,2))) 

SHORT
DeepSurv M1: R2: 0.01 RMSE: 396.0 C-idx: 0.53 MAE: 309.0 MAPE: 2008.0 NRMSE: 1.4
DeepSurv M5: R2: 0.01 RMSE: 397.0 C-idx: 0.53 MAE: 306.0 MAPE: 1957.0 NRMSE: 1.41


In [51]:
models = ['M1', 'M5']

print("LONG")
for mo in models:
    r = results[mo]['L']   
    RMSE = r['RMSE_test']
    R2 = r['R2_test']
    MAE = r['MAE_test']
    MAPE = r['MAPE_test']
    NRMSE = r['NRMSE_test']
    C_idx = r['c-test']

    print("DeepSurv {}: R2: {} RMSE: {} C-idx: {} MAE: {} MAPE: {} NRMSE: {}".format(mo,
                                                                          round(R2,2),
                                                                          round(RMSE,0),
                                                                          round(C_idx,2),
                                                                          round(MAE,0),
                                                                          round(MAPE,0),
                                                                          round(NRMSE,2))) 

LONG
DeepSurv M1: R2: 0.04 RMSE: 443.0 C-idx: 0.59 MAE: 326.0 MAPE: 1417.0 NRMSE: 0.59
DeepSurv M5: R2: -0.01 RMSE: 454.0 C-idx: 0.6 MAE: 342.0 MAPE: 1312.0 NRMSE: 0.61


In [52]:
models = ['M1', 'M5']

print("Combined")
for mo in models:
    r = results[mo]['Com']   
    RMSE = r['RMSE_test']
    R2 = r['R2_test']
    MAE = r['MAE_test']
    MAPE = r['MAPE_test']
    NRMSE = r['NRMSE_test']
    C_idx = r['c-test']

    print("DeepSurv {}: R2: {} RMSE: {} C-idx: {} MAE: {} MAPE: {} NRMSE: {}".format(mo,
                                                                          round(R2,2),
                                                                          round(RMSE,0),
                                                                          round(C_idx,2),
                                                                          round(MAE,0),
                                                                          round(MAPE,0),
                                                                          round(NRMSE,2))) 

Combined
DeepSurv M1: R2: 0.25 RMSE: 415.0 C-idx: 0.65 MAE: 316.0 MAPE: 1781.0 NRMSE: 0.9
DeepSurv M5: R2: 0.25 RMSE: 420.0 C-idx: 0.65 MAE: 320.0 MAPE: 1709.0 NRMSE: 0.91
